In [3]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from sklearn.metrics import average_precision_score, log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split

In [4]:
dirty_train = pd.read_csv("../../../datasets/dataset_0/dataset_0_train.csv").set_index("user_id")
dirty_test = pd.read_csv("../../../datasets/dataset_0/dataset_0_test.csv").set_index("user_id")
#dirty_test.drop("CHURN", axis=1, inplace=True)

In [5]:
def data_load(train_y):
    train_y["REGION"] = train_y["REGION"].astype("category")
    train_y["TENURE"] = train_y["TENURE"].astype("category")
    train_y["MONTANT"] = train_y["MONTANT"].astype("int32")
    train_y["FREQUENCE_RECH"] = train_y["FREQUENCE_RECH"].astype("int16")
    train_y["REVENUE"] = train_y["REVENUE"].astype("int32")
    train_y["FREQUENCE"] = train_y["FREQUENCE"].astype("int16")
    train_y["DATA_VOLUME"] = train_y["DATA_VOLUME"].astype("int32")
    train_y["ON_NET"] = train_y["ON_NET"].astype("int32")
    train_y["ORANGE"] = train_y["ORANGE"].astype("int16")
    train_y["TIGO"] = train_y["TIGO"].astype("int16")  # int16 probably is fine
    train_y["ZONE1"] = train_y["ZONE1"].astype("int16")
    train_y["ZONE2"] = train_y["ZONE2"].astype("int16")
    train_y["REGULARITY"] = train_y["REGULARITY"].astype("int8")
    train_y["FREQ_TOP_PACK"] = train_y["FREQ_TOP_PACK"].astype("int16")
    train_y["FULL_CALLS_SUM"] = train_y["FULL_CALLS_SUM"].astype("int32")
    train_y["TELE_NANS_SUM"] = train_y["TELE_NANS_SUM"].astype("int8")
    train_y["NONTELE_NANS_SUM"] = train_y["NONTELE_NANS_SUM"].astype("int8")
    train_y["FULL_CALLS_SUM_TO_REGULATIRY"] = train_y[
        "FULL_CALLS_SUM_TO_REGULATIRY"
    ].astype("float16")
    train_y["DATA_VOLUME_TO_REGULARITY"] = train_y["DATA_VOLUME_TO_REGULARITY"].astype(
        "float32"
    )
    train_y["TEN_REGULARITY_MEDIAN"] = train_y["TEN_REGULARITY_MEDIAN"].astype(
        "int16"
    )  # Update 27.09
    train_y["TEN_REGULARITY_MEAN"] = train_y["TEN_REGULARITY_MEAN"].astype(
        "float16"
    )  # Update 27.09
    train_y["REG_TEN_REGULARITY_MEDIAN"] = train_y["REG_TEN_REGULARITY_MEDIAN"].astype(
        "int16"
    )  # Update 27.09
    train_y["REG_TEN_REGULARITY_MEAN"] = train_y["REG_TEN_REGULARITY_MEAN"].astype(
        "float16"
    )  # Update 27.09

    train_y["MONTANT_TO_REVENUE"] = train_y["MONTANT_TO_REVENUE"].astype("float16")
    train_y["POPULARITY"] = train_y["POPULARITY"].astype("float16")

    if "CHURN" in train_y.columns:
        train_y["CHURN"] = train_y["CHURN"].astype("int8")
    return train_y

In [6]:
dirty_X = data_load(dirty_train)
dirty_y = dirty_X["CHURN"]
dirty_X.drop("CHURN", axis=1, inplace=True)

In [7]:
""" 5-split to mix train/valid/generalization valid """
outer_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [8]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [9]:
# 5x2 setup
def objective_LGB_dirty(trial):
    results = {}  # dict for record_evaluation callback
    split_iteration = {}  # dict for best iterations in each fold
    inner_valid_values = {}  # dict for valid score inside kfold
    outter_valid_values = {}  # dict for valid score outside kfold
    # simple boosting type gbdt, try another later
    hyperparameters = {
        "objective": "binary",
        "metric": "auc",
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0, log=True),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-3, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-3, 10.0),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 8, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.1, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 30),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 100),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 3, 100),
        "min_sum_hessian_in_leaf": trial.suggest_loguniform(
            "min_sum_hessian_in_leaf", 1e-5, 9e-1
        ),
        "feature_pre_filter": False,
        "verbose": -1,
    }
    # Split to 5 sfk manually
    for split_number, data_splits in enumerate(outer_splitter.split(dirty_X, dirty_y)):
        # In single fold split to train/valid manually
        train_inner_X, valid_inner_X, train_inner_y, valid_inner_y = train_test_split(
            dirty_X.reset_index(drop=True).loc[data_splits[0]],
            dirty_y.reset_index(drop=True).loc[data_splits[0]],
            test_size=0.2,
            random_state=1,
            stratify=dirty_y.reset_index(drop=True).loc[data_splits[0]],
        )

        train_set = lgb.Dataset(train_inner_X, label=train_inner_y)
        valid_set = lgb.Dataset(valid_inner_X, label=valid_inner_y)

        # Train
        lgb_model = lgb.train(
            params=hyperparameters,
            train_set=train_set,
            valid_names=["train", "eval"],
            num_boost_round=1000,
            valid_sets=[train_set, valid_set],
            verbose_eval=100,
            early_stopping_rounds=25,
            callbacks=[lgb.record_evaluation(results)],
        )

        split_iteration.update({f"{split_number}": lgb_model.best_iteration})

        inner_valid_values.update({f"{split_number}": results["eval"]["auc"][-1]})

        outter_valid_values.update(
            {
                f"{split_number}": roc_auc_score(
                    y_true=dirty_y.reset_index(drop=True).loc[data_splits[1]],
                    y_score=lgb_model.predict(
                        dirty_X.reset_index(drop=True).loc[data_splits[1]]
                    ),
                )
            }
        )

        print(
            f"Best iteration, split №{split_number} = {lgb_model.best_iteration}\n"
        )  #

    trial.set_user_attr("Best_iteration", split_iteration)

    trial.set_user_attr("Best_inner_valid_score", inner_valid_values)

    trial.set_user_attr("Outter_valid_score", outter_valid_values)

    mean_val = np.mean(list(outter_valid_values.values()))
    trial.set_user_attr("Mean_valid_score", mean_val)

    mean_std = np.std(list(outter_valid_values.values()))
    trial.set_user_attr("Std_valid_score", mean_std)

    trial.set_user_attr("Mean_iterations", np.mean(list(split_iteration.values())))

    print(f"AUC: {mean_val} +\- {mean_std}\n")

    return mean_val

In [10]:
# pruner_LGB_dirty = optuna.pruners.MedianPruner(n_startup_trials=4, n_warmup_steps=100, interval_steps=10)
study_LGB_dirty = optuna.create_study(direction="maximize")  # , pruner=pruner_LGB_dirty
study_LGB_dirty.optimize(objective_LGB_dirty, n_trials=35)

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=30 will be ignored. Current value: min_data_in_leaf=20


D:\Anaconda\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
D:\Anaconda\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 25 rounds
Early stopping, best iteration is:
[27]	train's auc: 0.931114	eval's auc: 0.931738
Best iteration, split №0 = 27

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=30 will be ignored. Current value: min_data_in_leaf=20


D:\Anaconda\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
D:\Anaconda\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 25 rounds
Early stopping, best iteration is:
[22]	train's auc: 0.931474	eval's auc: 0.931094
Best iteration, split №1 = 22

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=30 will be ignored. Current value: min_data_in_leaf=20


...

Training until validation scores don't improve for 25 rounds
[100]	train's auc: 0.932568	eval's auc: 0.931453
Early stopping, best iteration is:
[103]	train's auc: 0.932568	eval's auc: 0.931455
Best iteration, split №2 = 103

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=37 will be ignored. Current value: min_data_in_leaf=85


D:\Anaconda\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
D:\Anaconda\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 25 rounds
[100]	train's auc: 0.932677	eval's auc: 0.931877
Early stopping, best iteration is:
[102]	train's auc: 0.932677	eval's auc: 0.931877
Best iteration, split №3 = 102

[LightGBM] [Warning] min_data_in_leaf is set=85, min_child_samples=37 will be ignored. Current value: min_data_in_leaf=85


D:\Anaconda\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
D:\Anaconda\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 25 rounds
[100]	train's auc: 0.93252	eval's auc: 0.93151
Early stopping, best iteration is:
[99]	train's auc: 0.932519	eval's auc: 0.93151
Best iteration, split №4 = 99

AUC: 0.9314615290175026 +\- 0.00030662344410068923



In [11]:
study_LGB_dirty.best_trial.user_attrs

{'Best_iteration': {'0': 612, '1': 658, '2': 626, '3': 526, '4': 515},
 'Best_inner_valid_score': {'0': 0.9323130086303436,
  '1': 0.931618410112145,
  '2': 0.9315344996767767,
  '3': 0.9319483883982833,
  '4': 0.9315686346413842},
 'Outter_valid_score': {'0': 0.9315747292504074,
  '1': 0.9311876939477102,
  '2': 0.9319464760642422,
  '3': 0.9311944195210159,
  '4': 0.9318100081707856},
 'Mean_valid_score': 0.9315426653908322,
 'Std_valid_score': 0.0003107549595163912,
 'Mean_iterations': 587.4}

In [12]:
study_LGB_dirty.best_params

{'learning_rate': 0.015199128013671246,
 'lambda_l1': 0.03354600337508153,
 'lambda_l2': 2.6666379329607066,
 'min_gain_to_split': 1.0570957418922402,
 'num_leaves': 102,
 'feature_fraction': 0.47243339229594256,
 'bagging_fraction': 0.6140362329314972,
 'bagging_freq': 3,
 'min_child_samples': 35,
 'min_data_in_leaf': 94,
 'min_sum_hessian_in_leaf': 0.3068170651632342}

In [13]:
parameters_for_fit_LGB_dirty = {
    "num_threads": 8,
    "objective": "binary",
    "metric": "auc",
    "feature_pre_filter": False,
    **study_LGB_dirty.best_params,
}

In [14]:
"""2 additional splitters for stacking"""
rs2_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
rs3_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)

In [15]:
probs_LGB = pd.DataFrame(
    index=dirty_X.index,
    columns=["LGB_proba_outer", "LGB_proba_rs2", "LGB_proba_rs3", "LGB_proba_mean"],
)

In [16]:
for train_split, predict_split in outer_splitter.split(dirty_X, dirty_y):
    ###############
    # train_set = lgb.Dataset(data=dirty_X.reset_index(drop=True).loc[train_split],
    #                        label=dirty_y.reset_index(drop=True).loc[train_split])
    #
    # predict_set = lgb.Dataset(data=dirty_X.reset_index(drop=True).loc[predict_split],
    #                          label=dirty_y.reset_index(drop=True).loc[predict_split])
    ################
    po = lgb.train(
        parameters_for_fit_LGB_dirty,
        train_set=lgb.Dataset(
            data=dirty_X.reset_index(drop=True).loc[train_split],
            label=dirty_y.reset_index(drop=True).loc[train_split],
        ),
        verbose_eval=False,
        num_boost_round=int(
            np.rint(study_LGB_dirty.best_trial.user_attrs["Mean_iterations"])
        ),
    )
    probs_LGB.loc[
        dirty_X.reset_index().loc[predict_split, "user_id"], "LGB_proba_outer"
    ] = po.predict(data=dirty_X.reset_index(drop=True).loc[predict_split])
print("Outer splitter probs\n", probs_LGB["LGB_proba_outer"].head())

[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Info] Number of positive: 323189, number of negative: 1400049
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.046864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4034
[LightGBM] [Info] Number of data points in the train set: 1723238, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187548 -> initscore=-1.466025
[LightGBM] [Info] Start training from score -1.466025
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=3

In [17]:
for train_split, predict_split in rs2_splitter.split(dirty_X, dirty_y):
    po = lgb.train(
        parameters_for_fit_LGB_dirty,
        train_set=lgb.Dataset(
            data=dirty_X.reset_index(drop=True).loc[train_split],
            label=dirty_y.reset_index(drop=True).loc[train_split],
        ),
        verbose_eval=False,
        num_boost_round=int(
            np.rint(study_LGB_dirty.best_trial.user_attrs["Mean_iterations"])
        ),
    )
    probs_LGB.loc[
        dirty_X.reset_index().loc[predict_split, "user_id"], "LGB_proba_rs2"
    ] = po.predict(data=dirty_X.reset_index(drop=True).loc[predict_split])
print("rs2 splitter probs\n", probs_LGB["LGB_proba_rs2"].head())

[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Info] Number of positive: 323189, number of negative: 1400049
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4038
[LightGBM] [Info] Number of data points in the train set: 1723238, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187548 -> initscore=-1.466025
[LightGBM] [Info] Start training from score -1.466025
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=3

In [18]:
for train_split, predict_split in rs3_splitter.split(dirty_X, dirty_y):
    po = lgb.train(
        parameters_for_fit_LGB_dirty,
        train_set=lgb.Dataset(
            data=dirty_X.reset_index(drop=True).loc[train_split],
            label=dirty_y.reset_index(drop=True).loc[train_split],
        ),
        verbose_eval=False,
        num_boost_round=int(
            np.rint(study_LGB_dirty.best_trial.user_attrs["Mean_iterations"])
        ),
    )
    probs_LGB.loc[
        dirty_X.reset_index().loc[predict_split, "user_id"], "LGB_proba_rs3"
    ] = po.predict(data=dirty_X.reset_index(drop=True).loc[predict_split])
print("rs3 splitter probs\n", probs_LGB["LGB_proba_rs3"].head())

[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Info] Number of positive: 323189, number of negative: 1400049
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4029
[LightGBM] [Info] Number of data points in the train set: 1723238, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187548 -> initscore=-1.466025
[LightGBM] [Info] Start training from score -1.466025
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=3

In [19]:
probs_LGB["LGB_proba_mean"] = np.mean(probs_LGB[["LGB_proba_outer", "LGB_proba_rs2", "LGB_proba_rs3"]], axis=1)

In [20]:
probs_LGB.drop(["LGB_proba_outer", "LGB_proba_rs2", "LGB_proba_rs3"], axis=1, inplace=True)

In [21]:
probs_LGB.to_csv("LGB_dataset_0_train_OOF_clean.csv", index=False)

In [22]:
refit_lgb_dirty = lgb.train(
    parameters_for_fit_LGB_dirty,
    train_set=lgb.Dataset(dirty_X, label=dirty_y),
    verbose_eval=False,
    num_boost_round=int(np.rint(study_LGB_dirty.best_trial.user_attrs["Mean_iterations"])),
)

[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=35 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Info] Number of positive: 403986, number of negative: 1750062
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4104
[LightGBM] [Info] Number of data points in the train set: 2154048, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187547 -> initscore=-1.466026
[LightGBM] [Info] Start training from score -1.466026


In [23]:
dirty_test = data_load(dirty_test)

In [24]:
subm_dirty = refit_lgb_dirty.predict(dirty_test)

In [25]:
sdf_dirty = pd.DataFrame(columns=["user_id", "CHURN"])
sdf_dirty["user_id"] = dirty_test.index
sdf_dirty["CHURN"] = subm_dirty

In [26]:
sdf_dirty.to_csv("LGB_dataset_0_test_clean.csv", index=False)